In [1]:
# Copyright 2024 Forusone
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# RAG Engine in Vertex AI


In [2]:
# @title Install packages
%pip install --upgrade --user --quiet google-cloud-aiplatform \
                                      google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.7/137.7 kB 7.1 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [3]:
# @title Authentication to access to GCP

# To use markdown for output data from LLM
from IPython.display import display, Markdown

# Use OAuth to access the GCP environment.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

In [4]:
# @title Define constants
PROJECT_ID = "ai-hangsik"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
MODEL_NAME = "gemini-2.0-flash-001" # @param {type:"string"}

In [12]:
# @title Initialize Vertex AI

import os

import vertexai
from vertexai import rag
from vertexai.generative_models import GenerativeModel, Tool

# from google.genai.types import GenerateContentConfig, Retrieval, Tool, VertexRagStore
from google import genai


vertexai.init(project=PROJECT_ID, location=LOCATION)
client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

In [13]:
# @title Create a RAG Corpus
# Currently supports Google first-party embedding models
# https://cloud.google.com/vertex-ai/generative-ai/docs/embeddings/get-text-embeddings#supported-models

#EMBEDDING_MODEL = "publishers/google/models/text-embedding-004"
EMBEDDING_MODEL="publishers/google/models/text-multilingual-embedding-002"

rag_corpus = rag.create_corpus(
    display_name="my-rag-corpus",
    backend_config=rag.RagVectorDbConfig(
        rag_embedding_model_config=rag.RagEmbeddingModelConfig(
            vertex_prediction_endpoint=rag.VertexPredictionEndpoint(
                publisher_model=EMBEDDING_MODEL
            )
        )
    ),
)

### Check the corpus just created

In [14]:
rag.list_corpora()

ListRagCorporaPager<rag_corpora {
  name: "projects/ai-hangsik/locations/us-central1/ragCorpora/8715591178868752384"
  display_name: "my-rag-corpus"
  create_time {
    seconds: 1741054737
    nanos: 98597000
  }
  update_time {
    seconds: 1741054737
    nanos: 98597000
  }
  corpus_status {
    state: ACTIVE
  }
  vector_db_config {
    rag_managed_db {
    }
    rag_embedding_model_config {
      vertex_prediction_endpoint {
        endpoint: "projects/721521243942/locations/us-central1/publishers/google/models/text-embedding-005"
      }
    }
  }
}
rag_corpora {
  name: "projects/ai-hangsik/locations/us-central1/ragCorpora/2662753279682805760"
  display_name: "my-rag-corpus"
  create_time {
    seconds: 1741077278
    nanos: 256646000
  }
  update_time {
    seconds: 1741077278
    nanos: 256646000
  }
  corpus_status {
    state: ACTIVE
  }
  vector_db_config {
    rag_managed_db {
    }
    rag_embedding_model_config {
      vertex_prediction_endpoint {
        endpoint: "proje

In [15]:
# @title Import files from Google Cloud Storage

transformation_config = rag.TransformationConfig(
      chunking_config=rag.ChunkingConfig(
          chunk_size=512,
          chunk_overlap=100,
      ),
  )

# INPUT_GCS_BUCKET_LIST = ["gs://it_laws_kr/law_pdf/개인정보 보호법(법률)(제19234호)(20240315).pdf",
#         "gs://it_laws_kr/law_pdf/정보통신망 이용촉진 및 정보보호 등에 관한 법률(법률)(제20069호)(20240123).pdf",
#         ]

INPUT_GCS_BUCKET = (
    "gs://it_laws_kr/law_pdf/"
)

response = rag.import_files(
    corpus_name=rag_corpus.name,
    paths=[INPUT_GCS_BUCKET],
    transformation_config=transformation_config, # Optional
    max_embedding_requests_per_min=1000,  # Optional
)

In [16]:
rag.list_files(rag_corpus.name)

ListRagFilesPager<rag_files {
  gcs_source {
    uris: "gs://it_laws_kr/law_pdf/개인정보 보호법(법률)(제19234호)(20240315).pdf"
  }
  name: "projects/721521243942/locations/us-central1/ragCorpora/1901081992703770624/ragFiles/5382349881370330465"
  display_name: "개인정보 보호법(법률)(제19234호)(20240315).pdf"
  create_time {
    seconds: 1741137733
    nanos: 786425000
  }
  update_time {
    seconds: 1741137733
    nanos: 786425000
  }
  file_status {
    state: ACTIVE
  }
}
rag_files {
  gcs_source {
    uris: "gs://it_laws_kr/law_pdf/신용정보의 이용 및 보호에 관한 법률(법률)(제19234호)(20230915).pdf"
  }
  name: "projects/721521243942/locations/us-central1/ragCorpora/1901081992703770624/ragFiles/5382349893237000266"
  display_name: "신용정보의 이용 및 보호에 관한 법률(법률)(제19234호)(20230915).pdf"
  create_time {
    seconds: 1741137735
    nanos: 386739000
  }
  update_time {
    seconds: 1741137735
    nanos: 386739000
  }
  file_status {
    stat

### Optional: Perform direct context retrieval

In [17]:
rag_retrieval_config=rag.RagRetrievalConfig(
    top_k=3,  # Optional
    filter=rag.Filter(vector_distance_threshold=0.5)  # Optional
)

# Direct context retrieval
response = rag.retrieval_query(
    rag_resources=[
        rag.RagResource(
            rag_corpus=rag_corpus.name,
            # Optional: supply IDs from `rag.list_files()`.
            # rag_file_ids=["rag-file-1", "rag-file-2", ...],
        )
    ],
    text="개인정보 보호법에 대해서 설명해주세요.",
    rag_retrieval_config=rag_retrieval_config,
)
print(response)

# Optional: The retrieved context can be passed to any SDK or model generation API to generate final results.
# context = " ".join([context.text for context in response.contexts.contexts]).replace("\n", "")

contexts {
  contexts {
    source_uri: "gs://it_laws_kr/law_pdf/정보통신망 이용촉진 및 정보보호 등에 관한 법률(법률)(제20069호)(20240123).pdf"
    source_display_name: "정보통신망 이용촉진 및 정보보호 등에 관한 법률(법률)(제20069호)(20240123).pdf"
    text: "가. 해킹, 컴퓨터바이러스, 논리폭탄, 메일폭탄, 서비스거부 또는 고출력 전자기파 등의 방법\r\n나. 정보통신망의 정상적인 보호ㆍ인증 절차를 우회하여 정보통신망에 접근할 수 있도록 하는\r\n프로그램이나 기술적 장치 등을 정보통신망 또는 이와 관련된 정보시스템에 설치하는 방법\r\n8. 삭제<2015. 6. 22.>\r\n9. “게시판”이란 그 명칭과 관계없이 정보통신망을 이용하여 일반에게 공개할 목적으로 부호ㆍ\r\n문자ㆍ음성ㆍ음향ㆍ화상ㆍ동영상 등의 정보를 이용자가 게재할 수 있는 컴퓨터 프로그램이나\r\n기술적 장치를 말한다."
    score: 0.38450904998889845
  }
  contexts {
    source_uri: "gs://it_laws_kr/law_pdf/신용정보의 이용 및 보호에 관한 법률(법률)(제19234호)(20230915).pdf"
    source_display_name: "신용정보의 이용 및 보호에 관한 법률(법률)(제19234호)(20230915).pdf"
    text: "이 경우 요청을 받은 행정안전부장관은 특별한 사유가 없으면 그 요청에 따라야 한다.\r\n<개정 2013. 3. 23., 2014. 11. 19., 2017. 7. 26.>\r\n1. 「상법」 제64조 등 다른 법률에 

### [Preview] Create RAG Retrieval Tool

In [19]:
# Create a tool for the RAG Corpus
rag_retrieval_tool = Tool.from_retrieval(
    retrieval=rag.Retrieval(
        source=rag.VertexRagStore(
            rag_resources=[
                rag.RagResource(
                    rag_corpus=rag_corpus.name,  # Currently only 1 corpus is allowed.
                    # Optional: supply IDs from `rag.list_files()`.
                    # rag_file_ids=["rag-file-1", "rag-file-2", ...],
                )
            ],
            rag_retrieval_config=rag_retrieval_config,
        ),
    )
)

In [20]:
# @title Generate Content with Gemini using Rag Retrieval Tool
# Load tool into Gemini model
rag_gemini_model = GenerativeModel(
    MODEL_NAME,
    tools=[rag_retrieval_tool],
)

In [21]:
response = rag_gemini_model.generate_content("개인정보 보호법에 대해서 설명해주세요.")
display(Markdown(response.text))

제공된 자료에서는 개인정보 보호법에 대한 직접적인 설명은 찾을 수 없습니다. 하지만 관련 법률인 "정보통신망 이용촉진 및 정보보호 등에 관한 법률"과 "신용정보의 이용 및 보호에 관한 법률"에 대한 정보는 찾을 수 있습니다.

정보통신망 이용촉진 및 정보보호 등에 관한 법률에서는 해킹, 컴퓨터 바이러스, 논리 폭탄, 메일 폭탄, 서비스 거부 등의 방법을 통해 정보통신망을 공격하는 행위나, 정보통신망의 정상적인 보호·인증 절차를 우회하는 프로그램 또는 기술적 장치를 설치하는 행위 등에 대한 내용을 다루고 있습니다. 또한, 게시판에 대한 정의도 포함되어 있습니다.

신용정보의 이용 및 보호에 관한 법률에서는 소멸시효가 완성된 예금 및 보험금 등의 지급을 위한 경우나 금융거래계약의 변경 사유 발생 시 주민등록전산정보자료를 요청할 수 있다는 내용이 있습니다. 개인신용평가회사의 겸영업무에 대한 내용도 포함되어 있습니다.
